In [1]:
import pandas as pd
import os
from modules.backtest import single_crypto_backtest, backtestCalculator

In [2]:
train_start, train_end = '2021-01-02 00:00:00', '2021-09-30 00:00:00',
test_start, test_end = '2021-10-01 00:00:00', '2021-12-29 00:00:00'

file_names = ['predictions_15mins', 'predictions_30mins', 'predictions_1hour','predictions_1day', 'predictions_30days']

In [3]:
for file_name in file_names:
    cash = 1000000 # initial cash

    K = 0 # keep track of timestamps

    backtest_result_folder = 'backtest_result'

    port_values, dates_portfolio = [], [] # store portfolio values

    interval = file_name.split('_')[1]
    df = pd.read_csv(f'model_predictions/{file_name}.csv')
    # df['signal'] =  df['prediction'].apply(lambda x: 1 if x > 0 else 0)
    df = df[['datetime','close', 'prediction']]
    df['datetime'] = pd.to_datetime(df['datetime'])  # Convert 'datetime' column to datetime format
    df.set_index('datetime', inplace=True)  # Set 'datetime' as the index

    test_df = df.loc[test_start:test_end]

    print(interval)
    if interval == '15mins':
        test_df = test_df.resample('15T').last()
    elif interval == '30mins':
        test_df = test_df.resample('30T').last()
    elif interval == '1hour':
        test_df = test_df.resample('1H').last()
    elif interval == '1day':
        test_df = test_df.resample('1D').last()
    elif interval == '30days':
        test_df = test_df.resample('30D').last()
    else:
        print("Invalid interval. Choose from '15T', '30T', '1H', '1D', '30D'.")

    test_df.dropna(inplace=True)

    # iterator to keep track of timestamps

    dates = test_df.index.to_list() # available timesteps of backtest

    # Run backtest

    for j in range(len(dates) - 1):
        try:

            t, t_1 = dates[K], dates[K+1] # pick current time and the t+1 timestep

            testDf = test_df.loc[t:t_1, ['close']] # take close of t and t+1 timesteps

            testDf.rename(columns={"close":"ETHUSDT"}, inplace=True)

            # receive the value invested at (t)th time, value at (t+1)th time, balance left and shares held

            cash_invested, current_value, balance, shares = backtestCalculator(cash=cash, backtest_df=testDf, weights={"ETHUSDT": 1 if test_df.loc[t, "prediction"] > 0 else 0})

            # portfolio value

            portfolio = balance + current_value

            cash = portfolio

            port_values.append(cash)

            dates_portfolio.append(t_1)

            K = K + 1 # goes to next trading time
        except Exception as e:
            #print(interval)
            continue

    portfolio_df = pd.DataFrame({'Timestamp': dates_portfolio, 'Portfolio Value': port_values})
    # Check if the folder exists, and create it if it doesn't
    if not os.path.exists(backtest_result_folder):
        os.makedirs(backtest_result_folder)

    backtest_file_path = os.path.join(backtest_result_folder, f'backtest_{interval}.csv')
    portfolio_df.to_csv(backtest_file_path, index=False)
    

15mins
30mins
1hour
1day
30days
